In [1]:
import pandas as pd
import requests

## US state population 2022

In [2]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population")

In [3]:
population = tables[0][[("State or territory", "State or territory"), ('Census population[8][a]', 'July 1, 2022 (est.)')]].iloc[:52].drop(30).reset_index(drop=True)
population.columns = ["Territory", "Population"]
population.loc[51] = ["Puerto Rico", 3221789]
population.loc[52] = ["Northern Mariana Islands", 47329]
population.loc[53] = ["Guam", 153586]
population.Population = population.Population.astype("int")
population.to_csv("us_state_population.csv", index=False)
population

,Territory,Population
0,California,39029342
1,Texas,30029572
2,Florida,22244823
3,New York,19677151
4,Pennsylvania,12972008
5,Illinois,12582032
6,Ohio,11756058
7,Georgia,10912876
8,North Carolina,10698973
9,Michigan,10034113


## U.S. Cities

In [4]:
response = requests.get("https://github.com/kelvins/US-Cities-Database/raw/main/csv/us_cities.csv")
with open("us-cities.csv", "w") as out:
    out.write(response.text)

In [5]:
response = requests.get("https://github.com/plotly/datasets/raw/master/us-cities-top-1k.csv")
with open("us-cities-with-population.csv", "w") as out:
    out.write(response.text)

## Laser incidents in 2019

From https://www.faa.gov/about/initiatives/lasers/laws/.

In [6]:
response = requests.get("https://www.faa.gov/about/initiatives/lasers/laws/media/Laser_Report_2019_final.xlsx")
with open("laser_incidents_2019.xlsx", "wb") as xls_out:
    xls_out.write(response.content)

In [7]:
incidents = pd.read_excel("laser_incidents_2019.xlsx")

In [8]:
sorted(incidents["State"].unique())

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'DC',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Guam',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Marina Islands',
 'Maryland',
 'Massachusetts',
 'Mexico',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Puerto Rico',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Teas',
 'Tennessee',
 'Texas',
 'Utah',
 'VA',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']